# P-multigrid Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Plots

In [ ]:
# setup
mesh = Mesh2D(1.0, 1.0)
finebasis = TensorH1LagrangeBasis(5, 5, 2)
coarsebasis = TensorH1LagrangeBasis(3, 5, 2)
lagrangequadrature = true
ctofbasis = TensorH1LagrangeBasis(3, 5, 2, lagrangequadrature)

function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end

# fine grid diffusion operator
fineinputs = [
    OperatorField(finebasis, [EvaluationMode.gradient]),
    OperatorField(finebasis, [EvaluationMode.quadratureweights]),
]
fineoutputs = [OperatorField(finebasis, [EvaluationMode.gradient])]
finediffusion = Operator(diffusionweakform, mesh, fineinputs, fineoutputs)

# coarse grid diffusion operator
coarseinputs = [
    OperatorField(coarsebasis, [EvaluationMode.gradient]),
    OperatorField(coarsebasis, [EvaluationMode.quadratureweights]),
]
coarseoutputs = [OperatorField(coarsebasis, [EvaluationMode.gradient])]
coarsediffusion = Operator(diffusionweakform, mesh, coarseinputs, coarseoutputs)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# compute operator symbols
numberruns = 100
maxeigenvalues = zeros(numberruns, numberruns)

for i in 1:numberruns, j in 1:numberruns
    A = computesymbols(multigrid, [0.7], [1, 1], [2π*(i/numberruns), 2π*(j/numberruns)])
    eigenvalues = [abs(val) for val in eigvals(A)]
    maxeigenvalues[i, j] = max(eigenvalues...)
end

# plot
heatmap(maxeigenvalues, title="Multigrid Symbols", transpose=true)